## Read training grid locations from gt10a.shp file and get corresponding band information for the patch from the raster file

In [1]:
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona, random

driver = gdal.GetDriverByName('GTiff')
#rasterFileName = "/home/gadiraju/data/bl-slums/raw-img/MUL_mosaic_415.tif" #path to raster
rasterFileName1 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_415.tif"
dataset1 = gdal.Open(rasterFileName1)
rasterFileName2 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_415_NDBI.tif"
dataset2 = gdal.Open(rasterFileName2)
rasterFileName3 = "/scratch/slums/bl-slums/features/pan/haralick/PAN_mosaic_415-simple-50.tif"
dataset3 = gdal.Open(rasterFileName3)
rasterFileName4 = "/scratch/slums/bl-slums/raw-img/PAN_mosaic_415_edgeDensity.tif"
dataset4 = gdal.Open(rasterFileName4)
vector = fiona.open('/scratch/slums/bl-slums/gt/final-data/gt10e/gt10e.shp')
imggeotrans = dataset1.GetGeoTransform()
coordinates_list = []
count_numbers = [0]*3
#print count_numbers
actual_class = []
train_test = []
ids_list = []
for feat in vector:
    curr_class = feat['properties']['Cl']
    if curr_class >0:
        #print curr_class
        coordinates_list.append(feat['geometry']['coordinates'])
        count_numbers[curr_class-1]+=1
        actual_class.append(curr_class)
        train_test.append(feat['properties']['Type'])
        ids_list.append(feat['properties']['ID'])
bands1=[]
bands2=[]
bands3=[]
bands4=[]
data_all_bands = []
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize

transform = dataset1.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

print xOrigin, yOrigin, pixelWidth, pixelHeight

for i in range(8):
    bands1.append(dataset1.GetRasterBand(i+1)) 
    # data_all_bands.append(band.ReadAsArray(0,0,cols,rows).astype(np.float))
bands2.append(dataset2.GetRasterBand(1))
for i in range(8):
    bands3.append(dataset3.GetRasterBand(i+1))
bands4.append(dataset4.GetRasterBand(1))
points_list = coordinates_list #list of X,Y coordinates

#points_list = [(756073.458902 , 1456683.91481)]
train_images_list = [[],[],[]]
test_images_list = [[],[],[]]
test_IDs = []
for pt in range(len(points_list)):
    if pt%100 == 0 and pt>0:
        print 'Finished {} points\n'.format(pt)
    point = points_list[pt]
    cls = actual_class[pt]
    #print point, cls
    curr = np.zeros((18,40,40))-1
    curr = curr.astype(float)
    col = int((point[0] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[1] ) / pixelHeight)
    for k in range(8):
        data = bands1[k].ReadAsArray(col,row,40,40).astype(np.float)
        #print data[9,9]
        curr[k,:,:] = data
    data = bands2[0].ReadAsArray(col,row,40,40).astype(np.float)
    curr[8,:,:] = data
    for k in range(8):
        data = bands3[k].ReadAsArray(col,row,40,40).astype(np.float)
        #print data[9,9]
        curr[9+k,:,:] = data
    data = bands4[0].ReadAsArray(col,row,40,40).astype(np.float)
    curr[17,:,:] = data
    curr = curr.T
    if train_test[pt] == 1:
        train_images_list[cls-1].append(curr)
    else:
        test_images_list[cls-1].append(curr) 
        test_IDs.append(ids_list[pt])

756073.457553 1456685.87304 0.499284950117 0.499231422086
Finished 100 points

Finished 200 points

Finished 300 points

Finished 400 points

Finished 500 points

Finished 600 points

Finished 700 points

Finished 800 points

Finished 900 points

Finished 1000 points

Finished 1100 points

Finished 1200 points

Finished 1300 points

Finished 1400 points

Finished 1500 points

Finished 1600 points

Finished 1700 points

Finished 1800 points

Finished 1900 points

Finished 2000 points

Finished 2100 points

Finished 2200 points

Finished 2300 points

Finished 2400 points

Finished 2500 points

Finished 2600 points

Finished 2700 points

Finished 2800 points

Finished 2900 points

Finished 3000 points

Finished 3100 points

Finished 3200 points

Finished 3300 points

Finished 3400 points

Finished 3500 points

Finished 3600 points

Finished 3700 points

Finished 3800 points

Finished 3900 points

Finished 4000 points

Finished 4100 points

Finished 4200 points

Finished 4300 points



In [2]:
for k in range(len(train_images_list)):
    tmp_Y_1 = np.zeros((len(train_images_list[k]),len(train_images_list)))
    #print tmp_Y_1.shape
    tmp_Y_1[:,k] = 1
    tmp_Y_2 = np.zeros((len(test_images_list[k]),len(test_images_list)))
    tmp_Y_2[:,k] = 1
    curr_train = train_images_list[k]
    curr_train = np.asarray(curr_train)
    print 'Curr_train shape ={}'.format(curr_train.shape)
    curr_test = np.asarray(test_images_list[k])
    print curr_train.shape, curr_test.shape
    print curr_test.shape
    if k==0:
        trainX = curr_train
        testX = curr_test
        trainY = tmp_Y_1
        testY = tmp_Y_2
        print trainX.shape, trainY.shape, testX.shape, testY.shape
    else:
        trainX = np.vstack((trainX, curr_train))
        testX = np.vstack((testX, curr_test))
        trainY = np.vstack((trainY, tmp_Y_1))
        testY = np.vstack((testY, tmp_Y_2))

print trainX.shape, trainY.shape, testX.shape, testY.shape


Curr_train shape =(1014, 40, 40, 18)
(1014, 40, 40, 18) (235, 40, 40, 18)
(235, 40, 40, 18)
(1014, 40, 40, 18) (1014, 3) (235, 40, 40, 18) (235, 3)
Curr_train shape =(1242, 40, 40, 18)
(1242, 40, 40, 18) (202, 40, 40, 18)
(202, 40, 40, 18)
Curr_train shape =(1291, 40, 40, 18)
(1291, 40, 40, 18) (342, 40, 40, 18)
(342, 40, 40, 18)
(3547, 40, 40, 18) (3547, 3) (779, 40, 40, 18) (779, 3)


## Save the above files to disk so that we don't need to re-create them many times

In [4]:

import pickle

f = open('/scratch/slums/bl-slums/gt/final-pt-tr-3-X','w')
pickle.dump(trainX,f)
f.close()

f = open('/scratch/slums/bl-slums/gt/final-pt-tr-3-Y','w')
pickle.dump(trainY,f)
f.close()

f = open('/scratch/slums/bl-slums/gt/final-pt-te-3-X','w')
pickle.dump(testX,f)
f.close()

f = open('/scratch/slums/bl-slums/gt/final-pt-te-3-Y','w')
pickle.dump(testY,f)
f.close()